# Funciones para realizar chunking (parsing parcial) de textos en Español

La idea general:
Encontrar “trozos” sintácticos del texto (sobre todo Frases Nominales, FN) usando dos enfoques:

Con spaCy (doc.noun_chunks)

Con NLTK + una gramática de expresiones regulares sobre etiquetas POS

In [1]:
# Para importar el módulo con las funciones necesarias
import sys
sys.path.append(r"D:\Repos Github\AnaliticaTextual-Ejemplos") # la carpeta donde está el módulo

import utils # el módulo -> utils.py

In [2]:
from nltk.chunk import RegexpParser as RE_Parser
# RegexpParser: clase de NLTK que permite definir gramáticas de chunking con expresiones regulares sobre etiquetas POS.
import es_core_news_sm # procesador de lenguaje con segmentación de frases, tokenización, lematización, etiquetas POS y dependencias sintácticas. Carga el modelo en español de Spacy

In [3]:
# Cargamos modelo de Spacy
nlp = es_core_news_sm.load()

In [4]:
# Ruta del corpus
FILENAME=r'D:\Repos Github\AnaliticaTextual-Ejemplos\CORPUS\deportes\d1.txt'

In [5]:
texto = open(FILENAME, 'r', encoding='latin-1').read()
# Aquí NO se trocea por frases, sino que se pasa el documento entero a las funciones de chunking.

In [6]:
# noun chunks con spaCy

def Chunk_FN(texto):
   doc = nlp(texto) #procesa texto con spacy
   lista_FN = [chunk.text for chunk in doc.noun_chunks] 
    # doc.noun_chunks ->Es un generador de frases nominales que spaCy detecta automáticamente usando el árbol de dependencias.
    # [chunk.text for chunk in doc.noun_chunks] ->Extrae solo el texto de cada chunk nominal.
   return(lista_FN)    

In [7]:
# chunking con NLTK + gramática

def Chunk(texto):
  tagged1 = utils.Etiquetar(texto). split() # -> lista de strings "palabra/POS"
  tagged = [tuple(s.split('/')) for s in tagged1] # ransformar cada "palabra/POS" en una tupla ("palabra", "POS"), utilizamos "/" como elemento del split 
    #-> formato que espera NLTK para chunking: lista de tuplas (palabra, etiqueta_POS)
  #print(tagged) para comprobar que el proceso funciona, lo qujito para no confundirme cuando imprima 
  # Definir un chunk del tipo Frase Nominal (FN). 
  # Ejemplo: articulo* nombre* adjetivo*
  gramatica = '''                                                                                                              
    FN:                                                                                                                    
        {<DET>*(<PROPN|NOUN>)+<ADJ>*}
  
    '''
# <DET>* → cero o más determinantes: rtículos, demostrativos, posesivos, etc. (“el”, “la”, “este”, “su”, …)
# <PROPN|NOUN>)+ → uno o más:
    #PROPN = nombre propio (“Messi”, “Madrid”, “España”, …)
    #NOUN = nombre común (“delantero”, “equipo”, “partido”, …)
# <ADJ>* → cero o más adjetivos: “argentino”, “local”, “grande”, “histórico”, …

# Con esto atrapa secuencias tipo "el delantero”, “el delantero argentino”, “este joven delantero argentino”, “Messi”, “Messi argentino zurdo”
    
    
  chunker = RE_Parser(gramatica) # creas el analizador de chunks de NLTK
  Arbol = chunker.parse(tagged) # Aquí es donde se aplica la gramática. Produce un árbol sintáctico “parcial” (un nltk.Tree) donde:
    # algunos grupos de tokens están agrupados como FN
    # el resto queda como tokens sueltos
  matches =[] # Aquí guardarás todos los subárboles que sean frases nominales (FN).
  for subarbol in Arbol.subtrees():
        if subarbol.label() == 'FN': 
            matches.append(subarbol)
  return(matches)

In [8]:
def Subarbol_a_Lista(Chunks):
  lista =[]
  for c in Chunks:
    palabras = []
    for (palabra,_pos) in c:
        palabras.append(palabra) # sólo cogemos palabra, no pos
    lista.append(' '.join(palabras))
  return(lista)  
# Así quedaría (ejemplo) -> ["El delantero argentino", "los goles", "el equipo local", ...]


In [9]:
FN = Chunk_FN(texto)

In [10]:
print(FN)

['El golfista chileno', 'la segunda jornada', 'una tarjeta', '+1', 'la parte final', 'certamen', 'que', 'Texas', ', Estados Unidos', 'Joaquín Niemann', 'lo', 'El golfista chileno', 'el corte', 'fin de semana', 'el Charles Schwab Challenge del PGA Tour', 'El chileno', 'día', 'muy parejo', 'Texas', ', con mejor rendimiento', 'la primera parte', ', pero con una segunda ronda', 'que', 'una tarjeta acumulada de +1', 'El talagantino', 'varias horas', 'los resultados', 'los otros rivales', 'el corte', '+2', 'competencia', 'Niemann', 'dos birdies', '(hoyos', 'dos bogeys', '(banderas', 'su registro', 'su participación', 'Chile', 'El sueco Jonas Blixt', 'como líder', '-9', 'cuatro birdies', '(hoyos 1, 2, 12 y 13)', 'un águila', 'la bandera']


In [11]:
FN2 = Subarbol_a_Lista(Chunk(texto))

In [12]:
print(FN2)

['El golfista chileno', 'jornada', 'una tarjeta', '+1', 'la parte final', 'certamen', 'Texas', 'Estados Unidos', 'Joaquín Niemann', 'El golfista chileno', 'el corte', 'fin', 'semana', 'el Charles Schwab Challenge', 'PGA Tour', 'El chileno', 'día', 'parejo', 'Texas', 'rendimiento', 'parte', 'ronda', 'una tarjeta acumulada', '+1', 'El talagantino', 'varias horas', 'los resultados', 'los otros rivales', 'el corte', '+2', 'competencia', 'Niemann', 'birdies', 'hoyos', 'bogeys', 'banderas', 'su registro', 'su participación', 'partir', 'horas', 'Chile', 'El sueco Jonas Blixt', 'líder', '-9', 'birdies', 'hoyos', 'un águila', 'la bandera']


Es más moderno usar FN. FN2 es un proceso manual. Atkinson lo pone aquí para comparar ambos sistemas. 

spaCy ya viene con el modelo entrenado para:

detectar tokens

etiquetar POS

analizar dependencias

y a partir de eso sacar automáticamente las Frases Nominales (FN)

No tienes que:

definir una gramática ({<DET>*(<PROPN|NOUN>)+<ADJ>*}),

partir cadenas "palabra/POS",

usar NLTK ni RegexpParser,

ni pelearte con árboles.

Tú solo cargas el modelo (es_core_news_sm) y le pasas texto.
El “modelo” ya está ahí dentro; lo que no tienes que hacer es diseñar tú las reglas.

Por eso:

FN (spaCy) → enfoque moderno, sencillo, práctico.

FN2 (NLTK + gramática) → enfoque clásico, didáctico, pero más engorroso y desfasado.

Para todo lo que tú quieres aprender de Analítica Textual te vale de sobra con spaCy.
Lo otro úsalo solo como “arqueología” para entender cómo se hacía antes.

JFBA